In [1]:
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage , AIMessage , HumanMessage

In [5]:
load_dotenv()

True

In [2]:
current_dir = os.getcwd()  # Works in Jupyter and scripts
db_path = os.path.join(current_dir, "db", "chroma_db_with_metadata")

In [8]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
db = Chroma(persist_directory=db_path, embedding_function=embeddings)

In [19]:
query = "What does dracula fear the most?"

# 5. Set up the retriever to search for similar documents
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

# 6. Get relevant documents
results = retriever.invoke(query)

# 7. Display the results
#print("\nRelevant Documents Found")
#for i, doc in enumerate(results, 1):
#    print(f"\nDocument {i}:")
#    print(doc.page_content)
#    print("Source:", doc.metadata.get("source", "Unknown"))

In [20]:
combined_input = (
    "Here are some documents that might help answer the question: "
    + query
    + "\n\nRelevant Documents:\n"
    + "\n\n".join([doc.page_content for doc in results])
    + "\n\nPlease provide a rough answer based only on the provided documents. "
    "If the answer is not found in the documents, respond with 'I'm not sure'."
)

# Create a ChatOpenAI model (gpt-4o)
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# Define the messages to pass to the model
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content=combined_input),
]

# Get the model's response
result = model.invoke(messages)

# Display the response
print("\nGenerated Response")
print(result.content)


Generated Response
Based on the provided documents, it appears Dracula fears being banished from his tomb by the Wafer.
